In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense

from sklearn.preprocessing import StandardScaler
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error,accuracy_score,f1_score
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn.metrics import mean_squared_error,log_loss
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
df_user_data_train = pd.read_csv("train/user_data.csv")
df_train_submission = pd.read_csv("train/train_submissions.csv")
df_problem_data_train = pd.read_csv("train/problem_data.csv")
df_sample_sub = pd.read_csv("sample_submissions_wbscxqU.csv")
df_test_sub = pd.read_csv("test_submissions_NeDLEvX.csv")

In [3]:
from keras import backend as K

def mcor(y_true, y_pred):
     #matthews_correlation
     y_pred_pos = K.round(K.clip(y_pred, 0, 1))
     y_pred_neg = 1 - y_pred_pos
 
 
     y_pos = K.round(K.clip(y_true, 0, 1))
     y_neg = 1 - y_pos
 
 
     tp = K.sum(y_pos * y_pred_pos)
     tn = K.sum(y_neg * y_pred_neg)
 
 
     fp = K.sum(y_neg * y_pred_pos)
     fn = K.sum(y_pos * y_pred_neg)
 
 
     numerator = (tp * tn - fp * fn)
     denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
 
 
     return numerator / (denominator + K.epsilon())




def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))


In [4]:
df_problem_data_train.drop(['tags'],axis=1,inplace=True)
#df_problem_data_train.points.fillna(0,inplace=True)
df_problem_data_train.level_type.fillna('N',inplace=True)
df_user_data_train.country.fillna('Missing',inplace=True)

def postive_only(x):
    if(x<0):
        return(-1*x)
    else:
        return(x)

def get_points(x1):
    if(x1=="A"):
        return(500)
    elif(x1=='B'):
        return(1000)
    elif(x1=='C'):
        return(1500)
    elif(x1=='D'):
        return(2000)
    elif(x1=='E'):
        return(2500)
    elif(x1=='F'):
        return(3000)
    elif(x1=='G'):
        return(3500)
    elif(x1=='H'):
        return(4000)
    else:
        return(1000)

df_problem_data_train['points']=df_problem_data_train['level_type'].map(lambda x: get_points(x))
df_user_data_train['submission_count']=df_user_data_train['submission_count'].map(lambda x: postive_only(x))
df_user_data_train['problem_solved']=df_user_data_train['problem_solved'].map(lambda x: postive_only(x))
df_user_data_train['contribution']=df_user_data_train['contribution'].map(lambda x: postive_only(x))
df_user_data_train['follower_count']=df_user_data_train['follower_count'].map(lambda x: postive_only(x))
df_user_data_train['last_online_time_seconds']=df_user_data_train['last_online_time_seconds'].map(lambda x: postive_only(x))
df_user_data_train['max_rating']=df_user_data_train['max_rating'].map(lambda x: postive_only(x))
df_user_data_train['registration_time_seconds']=df_user_data_train['registration_time_seconds'].map(lambda x: postive_only(x))

#from sklearn.preprocessing import LabelEncoder
#lb = LabelEncoder()
#df_user_data_train['country'] = lb.fit_transform(df_user_data_train['country'])
#df_user_data_train['rank'] = lb.fit_transform(df_user_data_train['rank'])
#df_problem_data_train['level_type'] = lb.fit_transform(df_problem_data_train['level_type'])


In [5]:
df_user_data_train['problem_sub_ratio'] = df_user_data_train['problem_solved'] / df_user_data_train['submission_count']

df_user_data_train['last_online_time_seconds'] = df_user_data_train.last_online_time_seconds.max() - df_user_data_train['last_online_time_seconds']
df_user_data_train['last_online_time_minutes'] = df_user_data_train['last_online_time_seconds'] / 60
df_user_data_train['last_online_time_hours'] = df_user_data_train['last_online_time_seconds'] / (60*60)
df_user_data_train['last_online_time_days'] = df_user_data_train['last_online_time_seconds'] / (60*60*24)
df_user_data_train['last_online_time_months'] = df_user_data_train['last_online_time_seconds'] / (60*60*24*30)
df_user_data_train['last_online_time_years'] = df_user_data_train['last_online_time_seconds'] / (60*60*24*365)

df_user_data_train['registration_time_seconds'] = df_user_data_train.registration_time_seconds.max() - df_user_data_train['registration_time_seconds']
df_user_data_train['registration_time_minutes'] = df_user_data_train['registration_time_seconds'] / 60
df_user_data_train['registration_time_hours'] = df_user_data_train['registration_time_seconds'] / (60*60)
df_user_data_train['registration_time_days'] = df_user_data_train['registration_time_seconds'] / (60*60*24)
df_user_data_train['registration_time_months'] = df_user_data_train['registration_time_seconds'] / (60*60*24*30)
df_user_data_train['registration_time_years'] = df_user_data_train['registration_time_seconds'] / (60*60*24*365)

df_user_data_train['last_vs_reg_time_ratio'] = df_user_data_train['last_online_time_seconds']/(1+df_user_data_train['registration_time_seconds'])

df_user_data_train['max_rating_current_ratio'] = df_user_data_train['max_rating']/ (1+df_user_data_train['rating'])
df_user_data_train['current_rating_per_registration_time'] = df_user_data_train['rating']/(1+df_user_data_train['registration_time_days'])
df_user_data_train['Follower_vs_Rating'] = df_user_data_train['follower_count']/(1+df_user_data_train['rating'])
df_user_data_train['Rating_mul_Follower'] = df_user_data_train['rating'] * df_user_data_train['follower_count']
df_user_data_train['follower_vs_Contribution'] = df_user_data_train['follower_count']/(1+df_user_data_train['contribution'])



df_problem_data_train = pd.get_dummies(df_problem_data_train,columns=['level_type'])
df_user_data_train = pd.get_dummies(df_user_data_train,columns=['country','rank'])

In [17]:
df_train  = pd.merge(df_train_submission,df_user_data_train, how='left', on='user_id')
df_train = pd.merge(df_train,df_problem_data_train,how='left',on='problem_id')

df_test  = pd.merge(df_test_sub,df_user_data_train, how='left', on='user_id')
df_test = pd.merge(df_test,df_problem_data_train,how='left',on='problem_id')

X = df_train.drop(['attempts_range','user_id','problem_id'],axis=1).values
y = df_train.attempts_range.values

X_test = df_test.drop(['ID','user_id','problem_id'],axis=1).values

ss = StandardScaler()
X = ss.fit_transform(X)
X_test_sub = ss.transform(X_test)

dummy_y = np_utils.to_categorical(y)

#X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.33, random_state=42)

In [18]:
clf_RF =RandomForestClassifier()
y_pred_cross = cross_val_predict(clf_RF,X,y,cv=8,n_jobs=-1)
print(f1_score(y,y_pred_cross,average="weighted"))
clf_RF = RandomForestClassifier()
clf_RF.fit(X,y)
y_pred = clf_RF.predict(X_test_sub)
df_train['y_pred'] = y_pred_cross
df_test['y_pred'] = y_pred
X = df_train.drop(['attempts_range','user_id','problem_id'],axis=1).values
y = df_train.attempts_range.values

X_test = df_test.drop(['ID','user_id','problem_id'],axis=1).values

ss = StandardScaler()
X = ss.fit_transform(X)
X_test = ss.transform(X_test)
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.33, random_state=42)

0.444477874594


In [19]:
dummy_y

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.]])

In [20]:
X_train.shape
X_test.shape

(51248, 125)

In [21]:
model = Sequential()
model.add(Dense(1200, input_dim=X.shape[1], activation='relu'))
model.add(Dense(640, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(dummy_y.shape[1], activation='softmax'))

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[f1])

In [ ]:

model.fit(X_train, y_train, epochs=500, batch_size=10000,validation_data=(X_test, y_test))


Train on 104047 samples, validate on 51248 samples
Epoch 1/500
104047/104047 [==============================] - 0s - loss: 1.0633 - f1: 0.4994 - val_loss: 1.2017 - val_f1: 0.4772
Epoch 2/500
104047/104047 [==============================] - 0s - loss: 1.0623 - f1: 0.5017 - val_loss: 1.2044 - val_f1: 0.4700
Epoch 3/500
104047/104047 [==============================] - 0s - loss: 1.0586 - f1: 0.5010 - val_loss: 1.2097 - val_f1: 0.4366
Epoch 4/500
104047/104047 [==============================] - 0s - loss: 1.0572 - f1: 0.5011 - val_loss: 1.2205 - val_f1: 0.4969
Epoch 5/500
104047/104047 [==============================] - 0s - loss: 1.0576 - f1: 0.5014 - val_loss: 1.2100 - val_f1: 0.4337
Epoch 6/500
104047/104047 [==============================] - 0s - loss: 1.0549 - f1: 0.4990 - val_loss: 1.2173 - val_f1: 0.5008
Epoch 7/500
104047/104047 [==============================] - 0s - loss: 1.0519 - f1: 0.5016 - val_loss: 1.2106 - val_f1: 0.4774
Epoch 8/500
104047/104047 [==========================

104047/104047 [==============================] - 0s - loss: 0.9389 - f1: 0.5688 - val_loss: 1.3734 - val_f1: 0.4968
Epoch 65/500
104047/104047 [==============================] - 0s - loss: 0.9350 - f1: 0.5684 - val_loss: 1.3755 - val_f1: 0.4664
Epoch 66/500
104047/104047 [==============================] - 0s - loss: 0.9358 - f1: 0.5697 - val_loss: 1.3849 - val_f1: 0.4888
Epoch 67/500
104047/104047 [==============================] - 0s - loss: 0.9322 - f1: 0.5729 - val_loss: 1.4000 - val_f1: 0.4996
Epoch 68/500
104047/104047 [==============================] - 0s - loss: 0.9294 - f1: 0.5744 - val_loss: 1.3952 - val_f1: 0.4630
Epoch 69/500
104047/104047 [==============================] - 0s - loss: 0.9267 - f1: 0.5734 - val_loss: 1.3958 - val_f1: 0.5007
Epoch 70/500
104047/104047 [==============================] - 0s - loss: 0.9321 - f1: 0.5734 - val_loss: 1.3917 - val_f1: 0.4613
Epoch 71/500
104047/104047 [==============================] - 0s - loss: 0.9269 - f1: 0.5740 - val_loss: 1.385

104047/104047 [==============================] - 0s - loss: 0.8736 - f1: 0.5977 - val_loss: 1.5220 - val_f1: 0.4736
Epoch 128/500
104047/104047 [==============================] - 0s - loss: 0.8741 - f1: 0.5975 - val_loss: 1.5275 - val_f1: 0.4911
Epoch 129/500
104047/104047 [==============================] - 0s - loss: 0.8736 - f1: 0.5991 - val_loss: 1.5270 - val_f1: 0.4776
Epoch 130/500
104047/104047 [==============================] - 0s - loss: 0.8720 - f1: 0.5968 - val_loss: 1.5320 - val_f1: 0.4819
Epoch 131/500
104047/104047 [==============================] - 0s - loss: 0.8706 - f1: 0.5971 - val_loss: 1.5346 - val_f1: 0.4972
Epoch 132/500
104047/104047 [==============================] - 0s - loss: 0.8699 - f1: 0.5985 - val_loss: 1.5330 - val_f1: 0.4866
Epoch 133/500
104047/104047 [==============================] - 0s - loss: 0.8708 - f1: 0.5963 - val_loss: 1.5388 - val_f1: 0.5097
Epoch 134/500
104047/104047 [==============================] - 0s - loss: 0.8731 - f1: 0.5988 - val_loss

104047/104047 [==============================] - 0s - loss: 0.8501 - f1: 0.6087 - val_loss: 1.6180 - val_f1: 0.4832
Epoch 191/500
104047/104047 [==============================] - 0s - loss: 0.8475 - f1: 0.6086 - val_loss: 1.6198 - val_f1: 0.4942
Epoch 192/500
104047/104047 [==============================] - 0s - loss: 0.8400 - f1: 0.6123 - val_loss: 1.6235 - val_f1: 0.4890
Epoch 193/500
104047/104047 [==============================] - 0s - loss: 0.8439 - f1: 0.6077 - val_loss: 1.6337 - val_f1: 0.5019
Epoch 194/500
104047/104047 [==============================] - 0s - loss: 0.8452 - f1: 0.6073 - val_loss: 1.6268 - val_f1: 0.4732
Epoch 195/500
104047/104047 [==============================] - 0s - loss: 0.8455 - f1: 0.6077 - val_loss: 1.6118 - val_f1: 0.4597
Epoch 196/500
104047/104047 [==============================] - 0s - loss: 0.8423 - f1: 0.6089 - val_loss: 1.6257 - val_f1: 0.4798
Epoch 197/500
104047/104047 [==============================] - 0s - loss: 0.8403 - f1: 0.6108 - val_loss

104047/104047 [==============================] - 0s - loss: 0.8277 - f1: 0.6131 - val_loss: 1.7055 - val_f1: 0.4694
Epoch 254/500
104047/104047 [==============================] - 0s - loss: 0.8301 - f1: 0.6141 - val_loss: 1.6956 - val_f1: 0.4946
Epoch 255/500
104047/104047 [==============================] - 0s - loss: 0.8263 - f1: 0.6159 - val_loss: 1.7042 - val_f1: 0.4982
Epoch 256/500
104047/104047 [==============================] - 0s - loss: 0.8234 - f1: 0.6155 - val_loss: 1.6944 - val_f1: 0.4888
Epoch 257/500
104047/104047 [==============================] - 0s - loss: 0.8247 - f1: 0.6151 - val_loss: 1.6946 - val_f1: 0.4749
Epoch 258/500
104047/104047 [==============================] - 0s - loss: 0.8264 - f1: 0.6123 - val_loss: 1.6940 - val_f1: 0.4773
Epoch 259/500
104047/104047 [==============================] - 0s - loss: 0.8278 - f1: 0.6160 - val_loss: 1.6969 - val_f1: 0.4772
Epoch 260/500
104047/104047 [==============================] - 0s - loss: 0.8231 - f1: 0.6162 - val_loss

104047/104047 [==============================] - 0s - loss: 0.8144 - f1: 0.6201 - val_loss: 1.7448 - val_f1: 0.4907
Epoch 317/500
104047/104047 [==============================] - 0s - loss: 0.8141 - f1: 0.6198 - val_loss: 1.7307 - val_f1: 0.4870
Epoch 318/500
104047/104047 [==============================] - 0s - loss: 0.8122 - f1: 0.6210 - val_loss: 1.7597 - val_f1: 0.4851
Epoch 319/500
104047/104047 [==============================] - 0s - loss: 0.8136 - f1: 0.6202 - val_loss: 1.7413 - val_f1: 0.4703
Epoch 320/500
100000/104047 [===========================>..] - ETA: 0s - loss: 0.8143 - f1: 0.6181

In [ ]:
y_pred = model.predict(X_test_sub,batch_size=10000)

In [ ]:
y_pred_val = y_pred.argmax(1)


In [ ]:
df_sample_sub['ID'] = df_test.ID
df_sample_sub['attempts_range'] = y_pred_val
df_sample_sub.to_csv("Keras_with_Basic__FE.csv",index=False)